In [1]:
import pandas as pd
import numpy as np
import os
import ast
import re

# Read the prepared data file
prep_df = pd.read_csv("prep_movie_list_original.csv", sep=',', encoding='latin-1')
#prep_df.head(2)
#print(len(prep_df.index))

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
CV= CountVectorizer(max_features=5000, stop_words="english")
vector=CV.fit_transform(prep_df['listtags']).toarray()
#check random features name
#CV.get_feature_names()[1000]

In [3]:
tfidf=TfidfVectorizer(max_features=5000,analyzer='word',stop_words="english")
tfdf_features=tfidf.fit_transform(prep_df['listtags'])

from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(vector)
#prep_df[prep_df['title']=='DARK SKIES'].index[0]

sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x : x[1])[1:6] # key define second column basis sort
tfdf_similarity=cosine_similarity(tfdf_features)

In [4]:
#Recommender function
reco_list = []
def Recommend_Movies(movie):
    global reco_list
    try:
        movie_index=prep_df[prep_df['title']==movie].index[0]
        distance=tfdf_similarity[movie_index]
        
        movies_list=sorted(list(enumerate(distance)), reverse=True, key=lambda x : x[1])[1:11]

        if len(movies_list) > 0:
            for i in movies_list:
                #print(df8.iloc[i[0]].title)
                t=prep_df.iloc[i[0]].title
                
                reco_list.append(t)
        else:
            print('Movie '+ movie +' does not have any matching recommendation. Please try another one.')
    except:
        print('Movie '+ movie +' not present in Netflix. Please try another one.')

In [5]:
#User input section

'''
parameter 1:
    more than one word: movie title
'|' is the delimiter
parameter 2:
    single word: one genre of the movie
'|' is the delimiter
parameter 3:
    Cast member name 
'''
errorKey = 0
inp_param = 'JAWS||'
param1 = str(inp_param.split('|',1)[0])
param2 = inp_param.split('|',1)[1].split('|',1)[0]
param3 = inp_param.rsplit('|',1)[1]

Recommend_Movies(param1)
    
recolistdf = pd.DataFrame(reco_list)
recolistdf = recolistdf.rename(columns={0: 'title'})

#recolistdf = recolistdf.merge(q_movies[['title','demog_score']], on  = "title",how="left")
recolistdf = recolistdf.merge(prep_df[['title','cast','director','imdb_votes','keywords','genre','imdb_score','popularity_score','description','tags']], on  = "title",how="left")
recolistdf = recolistdf.drop_duplicates(subset = ['title'],keep = 'last').reset_index(drop = True)

recolistdf[['imdb_score']] = recolistdf[['imdb_score']].fillna(value=0)
recolistdf[['imdb_votes']] = recolistdf[['imdb_votes']].fillna(value=0)
recolistdf[['popularity_score']] = recolistdf[['popularity_score']].fillna(value=0)

#recolistdf['imdb_votes'] = recolistdf['imdb_votes'].astype(float)
recolistdf['imdb_score'] = recolistdf['imdb_score'].astype(float)
recolistdf['popularity_score'] = recolistdf['popularity_score'].astype(float)
recolistdf['composite_score'] = recolistdf['imdb_score']*0.6+recolistdf['popularity_score']*0.4
#recolistdf = recolistdf.drop(columns=['imdb_votes', 'imdb_score','popularity_score'])
recolistdf = recolistdf.sort_values(by=['composite_score','title'], ascending=False)

#recolistdf = recolistdf.loc[recolistdf['overview'].str.contains('fun',na=False)]
if len(param2) > 0:
    recolistdf = recolistdf.loc[recolistdf['genre'].str.contains(param2.lower())]
if len(param3) > 0:
    recolistdf = recolistdf.loc[recolistdf['cast'].str.lower().str.contains(param3.lower())]

recolistdf

#recolistdf.head(10)



,title,cast,director,imdb_votes,keywords,genre,imdb_score,popularity_score,description,tags,composite_score
2,THE STRONGHOLD,"['Gilles', 'Lellouche,', 'Karim', 'Leklou,', '...",CÃÂ©dric Jimenez,9422.0,"['police marseille, france ']","['', 'action', 'crime', 'thriller', 'european'...",7.5,120.881,"['Tired', 'of', 'the', 'small-time', 'grind,',...","[['', 'action', 'crime', 'thriller', 'european...",52.8524
6,TRAINING DAY,"['Denzel', 'Washington,', 'Ethan', 'Hawke,', '...",Antoine Fuqua,0.0,['neo-noir cholo barrio mexican american narco...,"['', 'drama', 'crime', 'action', '', 'thriller...",0.0,45.667,"['A', 'rookie', 'cop', 'with', 'one', 'day', '...","[['', 'drama', 'crime', 'action', '', 'thrille...",18.2668
3,JAWS,"['Roy', 'Scheider,', 'Robert', 'Shaw,', 'Richa...",Steven Spielberg,0.0,['shark cage animal horror fishing boat severe...,"['', 'adventure', 'thriller', 'horror', '', 'a...",0.0,44.988,"['When', 'an', 'insatiable', 'great', 'white',...","[['', 'adventure', 'thriller', 'horror', '', '...",17.9952
0,DARK SKIES,"['Keri', 'Russell,', 'Josh', 'Hamilton,', 'J.K...",Scott Stewart,75139.0,['fear alien horror supernatural '],"['', 'thriller', 'horror', 'scifi', 'documenta...",6.3,25.799,"['A', 'familyÃ¢\x80\x99s', 'idyllic', 'suburba...","[['', 'thriller', 'horror', 'scifi', 'document...",14.0996
1,PARANOIA,"['Liam', 'Hemsworth,', 'Gary', 'Oldman,', 'Amb...",Robert Luketic,0.0,['escape deception bodyguard paranoia based on...,"['', 'thriller', 'drama', '', 'drama', '']",0.0,29.299,"['Blackmailed', 'by', 'his', ""company's"", 'CEO...","[['', 'thriller', 'drama', '', 'drama', ''], [...",11.7196
5,SAFE HOUSE,"['Denzel', 'Washington,', 'Ryan', 'Reynolds,',...",Daniel Espinosa,0.0,['consulate football (soccer) stadium cape tow...,"['', 'thriller', 'action', '', 'thriller', 'ac...",0.0,21.920,"['Young', 'CIA', 'operative', 'Matt', 'Weston'...","[['', 'thriller', 'action', '', 'thriller', 'a...",8.7680
4,JAWS 2,"['Roy', 'Scheider,', 'Lorraine', 'Gary,', 'Mur...",Jeannot Szwarc,0.0,['tourism water skiing high-tension current ki...,"['', 'thriller', 'horror', '', 'thriller', 'ho...",0.0,16.792,"['Four', 'years', 'after', 'the', 'last', 'dea...","[['', 'thriller', 'horror', '', 'thriller', 'h...",6.7168


In [6]:
netflix_titles = pd.read_csv("10k_titles.csv")
for movie in recolistdf['title']:
    print(movie.lower().title())
    print(re.sub("[^A-Za-z ]","",str(netflix_titles.loc[netflix_titles['title'] == movie.lower().title()]['overview'].values)))
    print('\n')

The Stronghold
A police brigade working in the dangerous northern neighborhoods of Marseille where the level of crime is higher than anywhere else in France


Training Day
On his first day on the job as a narcotics officer a rookie cop works with a rogue detective who isnt what he appears


Jaws
When an insatiable great white shark terrorizes the townspeople of Amity Island the police chief an oceanographer and a grizzled shark hunter seek to destroy the bloodthirsty beast


Dark Skies
From the producers of Paranormal Activity Insidious and Sinister comes Dark Skies a supernatural thriller that follows a young family living in the suburbs As husband and wife Daniel and Lacey Barret witness an escalating series of disturbing events involving their family their safe and peaceful home quickly unravels When it becomes clear that the Barret family is being targeted by an unimaginably terrifying and deadly force Daniel and Lacey take matters in their own hands to solve the mystery of what is